<a href="https://colab.research.google.com/github/tahreemrasul/gemini_workshops/blob/main/router_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a router agent using Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/tahreemrasul/gemini_workshops/blob/main/router_agent.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/tahreemrasul/gemini_workshops/blob/main/router_agent.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>  

| Author |
| --- |
| [Tahreem Rasul](https://github.com/tahreemrasul/) |

### What is a Router Agent?

A router agent is an LLM that decides **which capability (tool)** should handle the user's request.  
Instead of forcing the model into a single function, we give it a *choice*:

- If the user asks a weather question → route to the **weather tool**  
- If the user asks a math question → route to the **calculator tool**  
- If neither tool makes sense → answer normally, without tools  

This is how more advanced agent frameworks (LangGraph, ADK, CrewAI) start to scale:
the agent **classifies intent**, then **dispatches** the request to the right tool or sub-agent.

In this notebook, we implement that idea using the [Google GenAI Python SDK](https://pypi.org/project/google-genai/), so you can see exactly how a router works under the hood without any frameworks.

---

### What We Will Build

We'll create:

1. **Two tools**  
   - `get_weather(city)`  
   - `calculate(expression)`

2. **A system instruction** that teaches Gemini:  
   *“Use the weather tool for weather queries, use the calculator tool for math, otherwise answer directly.”*

3. **A small agent loop** that:
   - sends the user message + tools to Gemini  
   - checks if the model wants to call a tool  
   - executes the tool in Python  
   - sends the result back to Gemini as a function response  
   - prints the final answer  

---

By the end of the notebook, you should know how

- to expose **multiple tools** to Gemini using the GenAI SDK  
- to write a **router-style system prompt**  
- the model chooses between tools based on intent  
- to implement a **manual function-calling agent loop**  
- routing is the foundation of more advanced multi-agent systems


Let's build it. 🚀


## Getting Started

### Install & Import Dependencies

In [ ]:
!pip install -U -q google-genai

In [ ]:
import os
import getpass
from typing import Dict

from google import genai
from google.genai import types

print("google-genai version:", genai.__version__)

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Get API Key

There are a few different ways of accessing the Gemini models in your applications. Most notably, you can do this through Vertex AI in Google Cloud Platform, or via the API method using Google AI Studio. We will be using the second approach.

Head over to https://aistudio.google.com/ and generate an API Key.
1. Click on **API Keys** on the left sidebar.
2. You should now see an option to **Create API Key**. Click on this.
3. A dialog box will open. You will be prompted to enter the name of the key. Enter a name of your choice. You also need to link this to a Google Cloud Project. From the dropdown of project selection, select **Create project**. You will be prompted to name the project, choose a name of your choice. Click on **Create Key**.
4. You should now see your created API key in the list. Click on the key and copy it. Paste it in the cell below.


In [ ]:
# 🔑 Paste your API key from Google AI Studio
API_KEY = "your-api-key"

from google import genai
client = genai.Client(api_key=API_KEY)

### Choose a model

In [ ]:
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

## Build the agent

### Step 1 - Define the weather and calculator tools


The weather tool will return the hard-coded values for a few cities. In a real app, this would call a live weather API. We'll add a second tool `calculate` that takes a simple math expression string, like `"2 + 3 * 4"` and outputs a JSON with `result` and an optional `error`.:


Then we describe it to Gemini using a **FunctionDeclaration** + **Tool**, so the
model knows:

- the tool name
- what it does
- what parameters it expects

In [ ]:
def get_weather_impl(city: str) -> Dict:
    """Mock weather function.

    In a real app, this would call an external API.
    Here we just return fake but structured data.
    """
    fake_temps = {
        "berlin": {"temperature_c": 17, "condition": "cloudy"},
        "london": {"temperature_c": 15, "condition": "windy"},
        "lahore": {"temperature_c": 32, "condition": "hot"},
        "islamabad": {"temperature_c": 28, "condition": "warm and clear"},
        "paris": {"temperature_c": 19, "condition": "light rain"},
    }

    city_key = city.lower()
    # return default fallback
    data = fake_temps.get(
        city_key,
        {"temperature_c": 22, "condition": "mild and unknown"},
    )
    return {
        "city": city,
        "temperature_c": data["temperature_c"],
        "condition": data["condition"],
    }

In [ ]:
from typing import Any, Dict

def calculate_impl(expression: str) -> Dict[str, Any]:
    """Very simple calculator for demo purposes.

    Evaluates a basic Python arithmetic expression.
    DO NOT use this pattern in production.
    """
    try:
        # extremely constrained eval – still just for demo
        result = eval(expression, {"__builtins__": {}}, {})
        return {"ok": True, "result": result}
    except Exception as e:
        return {"ok": False, "error": str(e)}

We will now describe the functions to the model.

In [ ]:
get_weather_decl = types.FunctionDeclaration(
    name="get_weather",
    description="Get the current (mock) weather for a given city.",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "City name, e.g. 'London' or 'Lahore'.",
            }
        },
        "required": ["city"],
    },
)

calculate_decl = types.FunctionDeclaration(
    name="calculate",
    description="Evaluate a simple math expression, like '2 + 3 * 4'.",
    parameters={
        "type": "object",
        "properties": {
            "expression": {
                "type": "string",
                "description": "Math expression in Python syntax, e.g. '2 + 3 * 4'.",
            }
        },
        "required": ["expression"],
    },
)

Now we bundle both tools into one Tool object the model can choose from.

In [ ]:
router_tool = types.Tool(
    function_declarations=[
        get_weather_decl,
        calculate_decl,
    ]
)

### Step 2 — Router-style system instruction

We want to bias Gemini to behave like a router:

- If query is weather-ish → call get_weather
- If query is math-ish → call calculate
- Otherwise → answer directly (no tool)

We will do this with a system instruction telling Gemini to route query requests to relevant tools.

In [ ]:
ROUTER_SYSTEM_INSTRUCTION = """
You are a routing agent.

Your job:
- Decide whether the user question is about WEATHER or about MATH.
- If it's about the weather in a specific city, call the `get_weather` function.
- If it's a math question (calculations, expressions), call the `calculate` function.
- If neither tool is appropriate, answer the question directly as a normal assistant.

Always prefer using a tool when it matches the user's intent.
When you use a tool, do NOT invent values – rely on the tool result.
"""


### Step 3 — Build the agent loop

We will now code a loop that does the following per turn:

1. Send user input + system instruction + tools.
2. If the model asks for a tool:
   - If `get_weather` → call `get_weather_impl`.
   - If `calculate` → call `calculate_impl`.
   - Send tool result back as a function response.
3. Print final answer.

Try with the following queries:
- What's the weather like in Berlin today? → should route to get_weather
- What is 12 * (3 + 4)? → should route to calculate
- Give me a tip for staying focused while studying. → answer directly, no tool
- whats 40 plus 80

In [ ]:
def run_router_agent():
    print("Router agent. Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")
        if user_input.strip().lower() in {"exit", "quit", "q"}:
            print("👋 Bye!")
            break

        # 1) Ask the model what to do, with tools + router instruction
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=[user_input],
            config=types.GenerateContentConfig(
                tools=[router_tool],
                system_instruction=ROUTER_SYSTEM_INSTRUCTION,
            ),
        )

        # If no function calls, just print the text
        if not response.function_calls:
            print("Agent (no tool):", response.text)
            print("-" * 60)
            continue

        # 2) Handle the first requested tool call
        tool_call = response.function_calls[0]
        print(f"[debug] Router chose tool: {tool_call.name} with args {tool_call.args}")

        tool_result = None

        if tool_call.name == "get_weather":
            city = tool_call.args.get("city", "")
            tool_result = get_weather_impl(city=city)

        elif tool_call.name == "calculate":
            expr = tool_call.args.get("expression", "")
            tool_result = calculate_impl(expression=expr)

        else:
            # Unknown tool – fallback: answer directly
            print("[warn] Unknown tool, falling back to direct answer.")
            print("Agent:", response.text)
            print("-" * 60)
            continue

        # Wrap the tool result as a function response part
        function_response_part = types.Part.from_function_response(
            name=tool_call.name,
            response=tool_result,
        )

        # 3) Second turn: send user + tool result back to the model
        followup_response = client.models.generate_content(
            model=MODEL_ID,
            contents=[
                user_input,
                function_response_part,
            ],
            config=types.GenerateContentConfig(
                tools=[router_tool],
                system_instruction=ROUTER_SYSTEM_INSTRUCTION,
            ),
        )

        print("Agent:", followup_response.text)
        print("-" * 60)

run_router_agent()